In [1]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
%matplotlib inline

In [2]:
SAMPLE_VIDEO_TRAIN = 10
SAMPLE_VIDEO_TEST  = 25           # Sample this many frames per video
BATCH = 128   # Process this many videos concurrently
LR = 1e-3
C = 157
PREFIX = "./dat/"
PRINT_EVERY = 1

In [3]:
label_dict = { 'c'+str(i).zfill(3) : i for i in range( 157 )}

In [4]:
fps_dict = { l.strip().split(' ')[0] : float( l.strip().split(' ')[1] ) for l in open( "video_fps.txt" ) }

In [5]:
def compute_mAP( data, labels, num_classes=4 ):

  mAP = 0.
  for i in range( num_classes ):
    # Sort the activations for class i by magnitude
    dd = data[ :, i ]
    ll = labels[:,i]
    idx = numpy.argsort( -dd )

    # True positives, False positives
    tp = (ll == 1).astype( numpy.int32 )
    fp = (ll == 0).astype( numpy.int32 )

    # Number of instances with label i
    num= numpy.sum( ll )

    # In case a class has been completely filtered by preprocessing:
    # For example c136: No frames before action that is to be predicted
    if num == 0:
      continue

    # Reorder according to the sorting
    tp = tp[idx]
    fp = fp[idx]

    tp = numpy.cumsum( tp ).astype( numpy.float32 )
    fp = numpy.cumsum( fp ).astype( numpy.float32 )

    prec = tp / (tp + fp)

    ap = 0.
    tmp = ll[idx]
    for j in range( data.shape[0] ):

      ap += tmp[j] * prec[j]
    ap /= num

    mAP += ap

  return mAP / num_classes

In [6]:
def fc( bottom, num_out, name, trainable = True ):
    shape = bottom.get_shape().as_list()
    if len(shape) > 2:
        bottom = tf.contrib.layers.flatten( bottom )

    num_in = bottom.get_shape().as_list()[-1]

    W = tf.get_variable( name + "/weights", shape = [num_in, num_out], initializer = tf.contrib.layers.xavier_initializer(), trainable = trainable )
    b = tf.get_variable( name + "/biases", shape = [num_out], initializer = tf.zeros_initializer(), trainable = trainable )

    return tf.matmul( bottom, W ) + b

In [7]:
X = tf.placeholder( "float", [None, 64*64*3] )
Y = tf.placeholder( "int32", [None, C] )

fc1 = fc( X, C, "fc1" )
pre = tf.nn.softmax_cross_entropy_with_logits( logits = fc1, labels = Y )
loss= tf.reduce_mean( pre )

In [8]:
optimizer  = tf.train.AdamOptimizer( learning_rate = LR, epsilon=1e-8 )
train_op = optimizer.minimize(loss)

In [9]:
conf = tf.ConfigProto(
      gpu_options = tf.GPUOptions( allow_growth = True ),
      device_count = { 'GPU': 1 }
    )

In [10]:
train_files = [l.strip() for l in open( "train.txt")]
test_files = [l.strip() for l in open( "test.txt")]

In [11]:
num_train = len(train_files)
num_test  = len(test_files)
test_vids = [l.split('|')[0] for l in test_files]

In [12]:
# path = '/Users/adrianhsu/Desktop/tensorflow-vgg/dat/'
# '001YG/001YG-000736.jpg'
# ALL_COLOR = 1 # 0 is grey scale
# frame = cv2.imread(path, ALL_COLOR)
# dmin = min( frame.shape[0], frame.shape[1] )
# ratio = 256./dmin
# # https://www.scivision.co/numpy-image-bgr-to-rgb/
# frame = frame[...,::-1] # bgr -> rgb
# frame = cv2.resize( frame, None, fx=ratio, fy=ratio )

In [13]:
# plt.imshow(rgb)
# plt.show()

In [14]:
# x = frame.shape[0]
# y = frame.shape[1]
# crop_size = 240

# x = (x - crop_size)//2
# y = (y - crop_size)//2
# crop = frame[ y : y+crop_size, x : x+crop_size, : ]

In [15]:
# plt.imshow(crop)
# plt.show()

In [16]:
# VGG_MEAN = numpy.array( [104, 117, 123] ,dtype='uint8')
# crop -= VGG_MEAN

In [17]:
# plt.imshow(crop)

In [18]:
total = 0
for file in train_files:
    f = file.split('|')[0]
    classes = file.split('|')[1].split(';')
    for c in classes:
        name = c.split(' ')[0]
        ts = int(c.split(' ')[1])
        te = int(c.split(' ')[2])
        total += te - ts + 1
print(total)

test_total = 0
for file in test_files:
    f = file.split('|')[0]
    classes = file.split('|')[1].split(';')
    for c in classes:
        name = c.split(' ')[0]
        ts = int(c.split(' ')[1])
        te = int(c.split(' ')[2])
        test_total += te - ts + 1
print(test_total)

1889
2053


In [48]:
path = '/Users/adrianhsu/Desktop/tensorflow-vgg/dat/'

count = 0
data = np.zeros(shape=(total, 12288))
labels = np.zeros(shape=(total, C))

for file in train_files:
    f = file.split('|')[0]
    classes = file.split('|')[1].split(';')
    for c in classes:
        name = c.split(' ')[0]
        ts = int(c.split(' ')[1])
        te = int(c.split(' ')[2])
        
        crop_size = 64
        max_num = len(os.listdir(path + f + '/'))
        for t in range(ts, te + 1):
            if t == 0:
                t += 1
            if t > max_num:
                break
            path_t = path + f + '/' + f + '-000' + str(t).zfill(3) + '.jpg'
#             print(path_t)
            ALL_COLOR = 1 # 0 is grey scale
            frame = cv2.imread(path_t, ALL_COLOR)
            print(path_t)
            dmin = min( frame.shape[0], frame.shape[1] )
            ratio = 256.0/dmin
            # https://www.scivision.co/numpy-image-bgr-to-rgb/
            frame = frame[...,::-1] # bgr -> rgb
            frame = cv2.resize( frame, None, fx=ratio, fy=ratio )
#             print(frame.shape)
            x = frame.shape[0]
            y = frame.shape[1]
            x = (x - crop_size)//2
            y = (y - crop_size)//2
            crop = frame[ x : x+crop_size, y : y+crop_size, : ]
            
            VGG_MEAN = np.array( [104, 117, 123] ,dtype='uint8')
            crop -= VGG_MEAN
            crop = crop.flatten()
#             print(crop.shape)
            
            # label
            lab = label_dict[name]
            data[count] = crop
            labels[count][lab] = 1

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000326.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000327.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000328.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000329.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000330.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000331.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000332.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000333.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000334.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000335.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000336.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000337.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000338.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000339.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000340

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000079.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000080.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000081.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000082.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000083.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000084.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000085.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000086.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000087.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000088.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000089.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000090.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000091.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000092.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000093

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000292.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000293.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000294.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000295.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000296.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000297.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000298.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000299.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000300.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000301.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000302.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000303.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000304.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000305.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000306

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000466.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000467.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000468.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000469.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000470.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000471.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000472.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000473.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000474.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000475.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000476.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000477.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000478.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000479.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000480

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000622.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000623.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000624.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000625.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000626.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000627.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000628.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000629.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000630.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000631.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000632.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000633.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000634.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000635.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00HFP/00HFP-000636

...

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000069.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000070.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000071.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000072.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000073.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000074.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000075.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000076.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000077.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000078.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000079.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000080.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000081.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000082.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000083

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000374.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000375.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000376.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000377.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000378.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000379.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000380.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000381.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000382.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000383.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000384.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000385.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000386.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000387.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000388

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000409.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000410.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000411.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000412.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000413.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000414.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000415.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000416.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000417.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000418.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000419.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000420.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000421.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000422.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000423

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000209.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000210.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000211.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000212.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000213.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000214.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000215.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000216.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000217.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000218.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000219.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000220.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000221.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000222.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000223

/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000343.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000344.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000345.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000346.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000347.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000348.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000349.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000350.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000351.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000352.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000353.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000354.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000355.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000356.jpg
/Users/adrianhsu/Desktop/tensorflow-vgg/dat/00MFE/00MFE-000357

In [55]:
with tf.Session( config = conf ) as sess:
    tf.global_variables_initializer().run()

    for e in range(  ):
        print("\nEpoch", str(e).zfill(3))

        # Train Phase.
#         random.shuffle( train_files )
        cnt_iteration = 0

        for start, end in list( zip( range( 0, total, BATCH ), range( BATCH, total+1, BATCH ) )):

            d, l = data[start:end], labels[start:end]
            tf_op, tf_loss = sess.run( [train_op, loss], feed_dict = { X : d, Y : l } )

    #         if cnt_iteration % PRINT_EVERY == 0:
            print("Train Phase\t", "Iteration", str(start).zfill(5), "\tLoss", tf_loss)
            cnt_iteration += 1
        
        


Epoch 000
Train Phase	 Iteration 00000 	Loss 0.0
Train Phase	 Iteration 00128 	Loss nan
Train Phase	 Iteration 00256 	Loss nan
Train Phase	 Iteration 00384 	Loss nan
Train Phase	 Iteration 00512 	Loss nan
Train Phase	 Iteration 00640 	Loss nan
Train Phase	 Iteration 00768 	Loss nan
Train Phase	 Iteration 00896 	Loss nan
Train Phase	 Iteration 01024 	Loss nan
Train Phase	 Iteration 01152 	Loss nan
Train Phase	 Iteration 01280 	Loss nan
Train Phase	 Iteration 01408 	Loss nan
Train Phase	 Iteration 01536 	Loss nan
Train Phase	 Iteration 01664 	Loss nan

Epoch 001
Train Phase	 Iteration 00000 	Loss 0.0
Train Phase	 Iteration 00128 	Loss nan
Train Phase	 Iteration 00256 	Loss nan
Train Phase	 Iteration 00384 	Loss nan
Train Phase	 Iteration 00512 	Loss nan
Train Phase	 Iteration 00640 	Loss nan
Train Phase	 Iteration 00768 	Loss nan
Train Phase	 Iteration 00896 	Loss nan
Train Phase	 Iteration 01024 	Loss nan
Train Phase	 Iteration 01152 	Loss nan
Train Phase	 Iteration 01280 	Loss nan
Tra

In [ ]:
# a = 100
# b = 15
# list(zip( range( 0, a, b), range( b, a + 1, b )))

In [ ]:
results  = np.zeros( (num_test, C) )
gt       = np.zeros( (num_test, C) )

test_range = list(zip( range( 0, num_test, BATCH ), range( BATCH, num_test+1, BATCH ) ))
print(test_range)

['.DS_Store', '001YG', '00HFP', '00MFE', '00N38', '00NN7']